In [1]:
from importlib import reload
from libs import io , cv2 , np

import preprocessing as pp
import roi as roi
import detection as detect
from classifier import H3T_Classifier
from numbers_classifier import H3T_Numbers_Classifier
import videoread as vr
from skimage import filters


In [8]:

reload(roi)
reload(detect)
reload(vr)



<module 'videoread' from 'd:\\IP\\project\\real\\full_pipeline\\videoread.py'>

In [3]:
# INIT

sign_imgs_corr = detect.get_corrleation_matrices("../dataset/corr_signs")


In [ ]:

classifier = H3T_Classifier()


In [ ]:
classifier.prepare_training_data("../dataset/numbers_training_set")
classifier.train("svm")


In [ ]:

classifier.save_trained_model()

In [ ]:

classifier.load_trained_model()


In [18]:
numbers_classifier = H3T_Numbers_Classifier()

In [ ]:
numbers_classifier.prepare_training_data("../dataset/numbers_training_set")
numbers_classifier.train("svm")

In [ ]:
numbers_classifier.save_trained_model()

In [19]:
numbers_classifier.load_trained_model()

In [ ]:
video_path = '../dataset/videos/IMG_50661.mp4'
video_frames_folder = '../dataset/video_frames'
vr.clear_folder(video_frames_folder)
video_frames = vr.extract_frames(video_path, video_frames_folder, fps=15)

In [ ]:
from skimage.feature import hog
image = io.imread("../speed_classification/road_sign_dataset/training_set/speed_50/50 (30).png")
resized_img = cv2.resize(image, (128, 128))
gray_image = pp.gray_image(resized_img)   
feature_vector , transformed_hog = hog(gray_image , visualize = True  , pixels_per_cell=(8 , 8) , cells_per_block=(4 , 4) , transform_sqrt=True)
pp.show_images([gray_image, transformed_hog])

In [54]:
# image = io.imread("../dataset/screenshots/3.jpg")
image = video_frames[-3]

In [140]:
# Load the image
image = io.imread("../dataset/tc/00173.jpg")


In [149]:

resized_img = cv2.resize(image, (1280, 720))

#pp.show_images([image], ["Input Image"])

cropped_img = cv2.hconcat([resized_img[:, :(resized_img.shape[1] // 3)]  , resized_img[:, 2 * (resized_img.shape[1] // 3):]])

# Convert the image to grayscale
gray_image = pp.gray_image(resized_img)

equalized_image = pp.HistogramEqualization(gray_image)

# Apply edge detection
edge_image = pp.LoGEdgeDetection(equalized_image)

# show pp images
#pp.show_images([resized_img, image, cropped_img], ["Input Image", "Cropped Image", "Edge Image"])


In [147]:

# Extract ROIs
rois = roi.extract_roi(edge_image , resized_img)

# print(rois[0].shape)
#show rois

# new_images = []
# for i in range(len(rois)):
#     #  rois[i] = pp.gray_image(rois[i])
#      new_images.append(pp.HistogramEqualization(rois[i]))

# pp.show_images(rois)

In [145]:
if (len(rois)  == 0):
    print("no rois")
else:
    detected_image_index = detect.detect_sign(rois, sign_imgs_corr)
    print(detected_image_index) 
    if detected_image_index != -1:
        
        # feature_vector , transformed_hog = hog(rois[detected_image_index] , visualize = True , channel_axis=2  , pixels_per_cell=(16, 16) , transform_sqrt=True)
        # pp.show_images([rois[detected_image_index], transformed_hog])

        # red_channel = colored_image[:, :, 0]
        # green_channel = colored_image[:, :, 1]
        # blue_channel = colored_image[:, :, 2]

        # new_image = red_channel.copy()

        # for i in range(new_image.shape[0]):
        #     for j in range(new_image.shape[1]):
        #         if (red_channel[i][j]  < 200 and green_channel[i][j] < 200 and blue_channel[i][j] < 200):
        #             new_image[i][j] = 255
        #         else:
        #             new_image[i][j] = 0

        new_image = pp.gray_image(rois[detected_image_index])

        

        
        new_image = new_image[ 30:100 , 25:61 ]

        #perform closing
        kernel = np.ones((2,2), np.uint8)
        new_image = cv2.dilate(new_image, kernel, iterations=3)
        new_image = cv2.erode(new_image, kernel, iterations=1)
        
        thrsholded_image = np.zeros_like(new_image)
        threshold = filters.threshold_otsu(new_image)
        thrsholded_image[new_image <= threshold] = 1
        thrsholded_image = cv2.resize(thrsholded_image , (14, 28)).astype(np.uint8)
        
        new_black_image = np.zeros((32, 32))
        new_black_image[2:2+thrsholded_image.shape[0] , 9:9+thrsholded_image.shape[1]] = thrsholded_image

        # # apply dilation
        # kernel = np.ones((2, 2), np.uint8)
        # new_black_image = cv2.dilate(new_black_image, kernel, iterations=1)

        
        # pp.show_images([colored_image, new_image , thrsholded_image , new_black_image])




        # gray_roi = pp.gray_image(rois[detected_image_index])
        
        # gray_roi = pp.HistogramEqualization(gray_roi)

        # new_image = gray_roi[ 20:100 , 15:110 ]

        # first_digit = new_image[: , 0:new_image.shape[0]//2]

        # pp.show_images([gray_roi , new_image , first_digit], ["sign_detected" , "new_image" , "first digit"]) 
        
        # print(gray_roi.shape)
        print(f'prediction is {numbers_classifier.predict(new_black_image)}')
    else:
        print("no sign detected")


0.2953190020479882
0.2909279864201648
0.2762933474763957
0.2890423727405617
0.2800940557080641
0.2860081830649733
[6]
0
prediction is ['3']
